In [1]:
# Regular EDA(exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

from tqdm import tqdm
import string
import statistics
import re
import joblib
import random

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Import the training set
df = pd.read_csv('/kaggle/input/dataverse-ipa/dataverse_2023/trainIPAdb_u.csv')
df.head(2)

,text,ipa
0,এরপরও তারা বকেয়া পরিশোধ করেনি।,eɾpɔɾo t̪ɐɾɐ bɔkeʲɐ poɾɪʃod̪ʱ kɔɾenɪ।
1,আগে সুইস ব্যাংকে জমা টাকার কোনো প্রতিবেদন প্রক...,ɐge suɪ̯s bɛŋke ɟɔmɐ tɐkɐɾ kono pɾot̪ɪbed̪ɔn p...


In [3]:
# See a random sample from the training set 
rand_index = np.random.randint(0, len(df))
print(f'Text = {df.text[rand_index]}\nIPA = {df.ipa[rand_index]}')

Text = পানি উন্নয়ন বোর্ডের তত্ত্বাবধানে ও বিশ্বব্যাংকের অর্থায়নে বেড়িবাঁধ নির্মাণ প্রকল্পে ওই জমি অধিগ্রহণ করা হয়েছে।
IPA = pɐnɪ unnɔʲon boɾdeɾ t̪ɔt̪t̪ɐbɔd̪ʱɐne o bɪʃʃobɛŋkeɾ ɔɾt̪ʰɐʲone beɽɪbɐ̃d̪ʱ nɪɾmɐn pɾokɔlpe oɪ̯ ɟomɪ od̪ʱɪgɾohon kɔɾɐ hoʲecʰe।


In [4]:
new_df = pd.read_csv('/kaggle/input/dataverse-ipa/new_train.csv')
new_df.head(3)

,text,ipa
0,আমি বলেন সাত আদালতের।,ɐmɪ bɔlen ʃɐt̪ ɐd̪ɐlɔt̪eɾ।
1,তুমি করা চুরানব্বই রাখে।,t̪umɪ kɔɾɐ cuɾɐnɔbboɪ ɾɐkʰe।
2,আমরা স্বীকার আটান্ন উপস্থিত।,ɐmɾɐ ʃɪkɐɾ ɐtɐnno upost̪ʰɪt̪।


In [5]:
test_df = pd.read_csv('/kaggle/input/dataverse-ipa/dataverse_2023/testData.csv')
test_df.head(3)

,row_id_column_name,text
0,0,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...
1,1,এ নিয়ে বিবাদে ২০১৫ সালের ২ জুন রাত সাড়ে ১১টায় ...
2,2,আজ থেকে ১৪ বছর আগে তিনি চলে গেছেন না ফেরার দেশে।


In [6]:
# Make a function
def make_library(df):
    '''
    
    '''
    library = {}
    texts = df.text
    ipas = df.ipa
    for i in tqdm(range(len(df))):
        for (v1, v2) in zip(re.split(r'\s+|(?=।)', texts[i]), re.split(r'\s+|(?=।)', ipas[i])):
            library[v1] = v2
    return library

In [19]:
lib = make_library(df)
# lib.update(make_library(new_df))

100%|██████████| 21999/21999 [00:00<00:00, 49773.54it/s]


In [8]:
print(list(lib.items())[:10])
len(lib)

[('এরপরও', 'eɾpɔɾo'), ('তারা', 't̪ɐɾɐ'), ('বকেয়া', 'bɔkeʲɐ'), ('পরিশোধ', 'poɾɪʃod̪ʱ'), ('করেনি', 'kɔɾenɪ'), ('।', '।'), ('আগে', 'ɐge'), ('সুইস', 'suɪ̯s'), ('ব্যাংকে', 'bɛŋke'), ('জমা', 'ɟɔmɐ')]


34903

In [9]:
from transformers import AutoTokenizer
model_id = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
# Later, to load the model back
loaded_model = joblib.load('/kaggle/input/model1/model (1).pkl')
loaded_model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [ ]:
# Remove English alphanumeric values
alpha_pat = "[a-zA-z0-9০-৯()]"
test_df["text"] = test_df["text"].str.replace(alpha_pat, "", regex=True)

In [11]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model=loaded_model,tokenizer=tokenizer, device=0)

In [12]:
texts = "সুন্দর সুন্দরমনা আমি তোমাকে ভালবাসি।"
# texts = "বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশনাল ডাইরেক্টরগেমস অ্যান্ড স্পোর্টস ডিপার্টমেন্ট ওয়ালটন।"
# texts = 'আমি তোমাকে মন।'
ipas = pipe(texts, max_length=128, batch_size=16)
print(ipas)

[{'generated_text': 'ʃund̪ɔɾ ʃund̪ɔɾmonɐ ɐmɪ t̪omɐke bɦɐlobɐʃɪ।'}]


In [14]:
def calculate_ipa(txt):
    '''
    Converts a sentence to IPA using t5 model.
    '''
    ipas = pipe(txt, max_length=128, batch_size=16)
    return ipas[0]['generated_text']

def word_to_ipa(word):
    '''
    Converts a Word to IPA using t5 model.
    '''
    word += '।'
    ipas = pipe(word, max_length=128, batch_size=16)
    return ipas[0]['generated_text'][:-1]

In [15]:
# Check the function
word_to_ipa('তারা')

't̪ɐɾɐ'

In [62]:
def contains_bengali_digit(text):
    '''
    Is there a bengali digit in text
    '''
    return bool(re.search(r'[০-৯]', text))

In [63]:
def ipa_converter(text):
    '''
    Converts a sentence to IPA
    '''
    ipa = ''
    #for value in re.split(r'\s+|(?=।)', text):
    words = re.split(r'\s+|(?<=[()।])|(?=[()।])', text)
    
    # Filter out empty strings and extra spaces from the result
    words = [word.strip() for word in words if word.strip()]
    for value in words:
        if value in lib:
            ipa += lib[value]
            ipa += ' '
        elif contains_bengali_digit(value):
            ipa += value
            ipa += ' '
        else:
            # ipa += 'UNKey '
            ipa += word_to_ipa(value)
            ipa += ' '
            
    pattern = r'\s+(?=।)|(?<=।)\s+'
    cln_ipa = re.sub(pattern, '', ipa)
    
    # Replace ' (' with '('
    cln_ipa = re.sub(r'\(\s+', '(', cln_ipa)

    # Replace ') ' with ')'
    cln_ipa = re.sub(r'\s+\)', ')', cln_ipa)
    return cln_ipa

In [20]:
lib['বাংলাদেশ']

'bɐŋlɐd̪eʃ'

In [22]:
rand_index = np.random.randint(0, 1000)
print(rand_index)
print(f'Pred = {ipa_converter(test_df.text[rand_index])}\nTrue = {test_df.text[rand_index]}')

875
Pred = kɾɪs geɪ̯l tɪʃo o bɾendɔn mɛkkɐlɐm tɪnsɪ ɾɐn nɪʲe ɔpoɾɐɟɪt̪o t̪ʰeke lɔɽɐɪ̯ cɐlɪʲe ɟɐccʰen।
True = ক্রিস গেইল ৫১ ও ব্রেন্ডন ম্যাককালাম ২৩ রান নিয়ে অপরাজিত থেকে লড়াই চালিয়ে যাচ্ছেন।


# **Need to handle Exceptional Cases**
1. Numeric Words (ie. ০-৯)
2. Bracket

In [40]:
# Add brackets is the library
lib['('] = '('
lib[')'] = ')'

In [71]:
# See a random sample from the test set 
rand_index = np.random.randint(0, len(test_df))
print(f'Text = {test_df.text[rand_index]}')

Text = অপর ম্যাচে রিসা আক্তারের হ্যাটট্রিকের সুবাদে নড়াইলকে ৮-১ গোলে হারায় রংপুর।


In [66]:
import re

# Input sentence
text = "মেয়েদের আনইভেন বার (রাত ১টা ২১)ভারোত্তোলনমেয়েদের ৭৫ কেজি ফাইনাল (রাত ১২টা ৩০ ও ৪টা)ডাইভিংমেয়েদের ৩ মিটার স্প্রিং বোর্ড (রাত ১টা)।"

# Split the sentence using a regular expression to keep spaces, periods, and parentheses
words = re.split(r'\s+|(?<=[()।])|(?=[()।])', text)

# Filter out empty strings and extra spaces from the result
words = [word.strip() for word in words if word.strip()]

# Print the words
print(words)

['মেয়েদের', 'আনইভেন', 'বার', '(', 'রাত', '১টা', '২১', ')', 'ভারোত্তোলনমেয়েদের', '৭৫', 'কেজি', 'ফাইনাল', '(', 'রাত', '১২টা', '৩০', 'ও', '৪টা', ')', 'ডাইভিংমেয়েদের', '৩', 'মিটার', 'স্প্রিং', 'বোর্ড', '(', 'রাত', '১টা', ')', '।']


In [67]:
ipa_converter(text)

'mejed̪eɾ ɐnɪbɦen bɐɾ (ɾɐt̪ ১টা ২১) bɦɐɾot̪t̪olɔnmesed̪eɾ ৭৫ keɟɪ pʰɐɪ̯nɐl (ɾɐt̪ ১২টা ৩০ o ৪টা) dɐɪ̯bɦɪŋmejed̪eɾ ৩ mɪtɐɾ slɪŋ boɾd (ɾɐt̪ ১টা)।'

In [72]:
filt_cond = ["১-৯", 'A-Za-z0-9']
# Filtering text samples that contain English alphanumeric values
filt = test_df[lambda x: x["text"].str.contains("[()]")]

print(f'Length of the Df = {filt.shape}')

Length of the Df = (1460, 2)


In [74]:
filt.text[0]

'বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশনাল ডাইরেক্টরগেমস অ্যান্ড স্পোর্টস ডিপার্টমেন্ট ওয়ালটন।'

In [75]:
ipa_converter(filt.text[0])

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'bɪʃeʃ ot̪ɪt̪ʰɪ ephem ɪkbɐl bɪn ɐnoʷɐɾ (dɔn) ɛdɪʃonɐl dɐɪ̯ɾektɔɾgemɔʃ ɛnd spoɾts dɪpɐɾtoment oʷɐltɔn।'

## Seems ok for the moment. Now Just Make a submission File

In [76]:
tqdm.pandas()
test_df['ipa'] = test_df['text'].progress_apply(ipa_converter)

  0%|          | 0/27228 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 27228/27228 [1:06:30<00:00,  6.82it/s]


In [78]:
test_df.head()

,row_id_column_name,text,ipa
0,0,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...,bɪʃeʃ ot̪ɪt̪ʰɪ ephe̯m ɪkbɐl bɪn ɐnoʷɐɾ (dɔn) ɛ...
1,1,এ নিয়ে বিবাদে ২০১৫ সালের ২ জুন রাত সাড়ে ১১টায় ...,e nɪʲe bɪbɐd̪e ২০১৫ ʃɐleɾ ২ ɟun ɾɐt̪ ʃɐɽe ১১টা...
2,2,আজ থেকে ১৪ বছর আগে তিনি চলে গেছেন না ফেরার দেশে।,ɐɟ t̪ʰeke ১৪ bɔcʰoɾ ɐge t̪ɪnɪ cɔle gɛcʰen nɐ p...
3,3,নিহত ব্যক্তি কুতপালং টালের ই-২ ব্লকের আবুল বাছ...,nɪhɔt̪o bɛkt̪ɪ kut̪pɐlɔŋ tɐleɾ ই-২ blokeɾ ɐbul...
4,4,সংক্ষিপ্ত স্কোরশ্রীলংকা প্রথম ইনিংস ৪৮২ (করুনা...,ʃɔŋkkʰɪpt̪o skoɾsɾolɔŋkɐ pɾot̪ʰom ɪnɪŋʃo ৪৮২ (...


In [79]:
test_df.to_csv('submission.csv', index=False)

In [83]:
# See a random sample from the test set 
rand_index = np.random.randint(0, len(test_df))
print(f'Text = {test_df.text[rand_index]}\nIpa = {test_df.ipa[rand_index]}')

Text = রক্তের মধ্যে গুলির খোঁসা পড়ে রয়েছে।
Ipa = ɾɔkt̪eɾ mod̪d̪ʱe gulɪɾ khõʃɐ pɔɽe ɾoʲecʰe।


In [84]:
lenli = []
for i in range(len(test_df)):
    l1 = len(test_df['text'][i].split())
    l2 = len(test_df['ipa'][i].split())
    lenli.append([l1, l2])

In [85]:
lenlisrt = sorted(lenli, key= lambda x:x[1])

In [87]:
lenlisrt[:10]

[[4, 2],
 [3, 3],
 [3, 3],
 [3, 3],
 [3, 3],
 [3, 3],
 [3, 3],
 [3, 3],
 [3, 3],
 [3, 3]]

In [88]:
sdf = test_df.drop(['text'], axis=1)
sdf.head(2)

,row_id_column_name,ipa
0,0,bɪʃeʃ ot̪ɪt̪ʰɪ ephe̯m ɪkbɐl bɪn ɐnoʷɐɾ (dɔn) ɛ...
1,1,e nɪʲe bɪbɐd̪e ২০১৫ ʃɐleɾ ২ ɟun ɾɐt̪ ʃɐɽe ১১টা...


In [89]:
print(sdf.shape)
sdf.to_csv('/kaggle/working/sub.csv', index=False)

(27228, 2)
